# DEMO 

Read in the intake catalogues from the input to bias correction



#### required packages

In [1]:
import intake
import xarray as xr
from matplotlib import pyplot as plt

import glob
import pathlib
import traceback
from datetime import datetime

from xclim.indices import (
    keetch_byram_drought_index,
    griffiths_drought_factor,
    mcarthur_forest_fire_danger_index
)


#### start a local Dask client

In [2]:
from dask.distributed import Client
client = Client()
client

2024-06-12 18:41:06,194 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-06-12 18:41:06,197 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-06-12 18:41:06,204 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-cpu-bdw-0032.gadi.nci.org.au/5093/proxy/8787/status,
Dashboard: /node/gadi-cpu-bdw-0032.gadi.nci.org.au/5093/proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41677,Workers: 14
Dashboard: /node/gadi-cpu-bdw-0032.gadi.nci.org.au/5093/proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:37867,Total threads: 1
Dashboard: /node/gadi-cpu-bdw-0032.gadi.nci.org.au/5093/proxy/34365/status,Memory: 0 B
Nanny: tcp://127.0.0.1:46745,


#### ignore warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Obtain the desired catalogues of the simulations to processe

In [4]:
catalogue_path='/g/data/ia39/catalogues'
mRuns = sorted(glob.glob(catalogue_path+'/*ssp370*.json'))
print(len(mRuns))

13


In [5]:
# test loading all the catalogues of the data
for file in mRuns: 
    print(file)
    cat1=intake.open_esm_datastore(file)

/g/data/ia39/catalogues/AGCD-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_BOM_ACCESS-ESM1-5_ssp370_r6i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_BOM_CESM2_ssp370_r11i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_BOM_CMCC-ESM2_ssp370_r1i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_BOM_EC-Earth3_ssp370_r1i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_BOM_MPI-ESM1-2-HR_ssp370_r1i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_BOM_NorESM2-MM_ssp370_r1i1p1f1_BARPA-R_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_CSIRO_ACCESS-CM2_ssp370_r4i1p1f1_CCAM-v2203-SN_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_CSIRO_ACCESS-ESM1-5_ssp370_r6i1p1f1_CCAM-v2203-SN_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_CSIRO_CESM2_ssp370_r11i1p1f1_CCAM-v2203-SN_v1-r1_day.json
/g/data/ia39/catalogues/AGCD-05i_CSIRO_CMCC-ESM2_ssp370_r1i1p1f1_CCAM-v2203-SN_v1-r1_day.json
/g/data/ia39/

# Process one ensemble 

In [117]:
# From one catalogue list save variables
mindex=12  # run to process
mindex=7
cat=intake.open_esm_datastore(mRuns[mindex])
mtmp=mRuns[mindex].split('/')
model_name=mtmp[5].split('.')
print(model_name)
print(cat.unique()['variable']) #unique()['variable']
tmp=model_name[0].split('_'); ACS_model=tmp[1]

['AGCD-05i_CSIRO_ACCESS-CM2_ssp370_r4i1p1f1_CCAM-v2203-SN_v1-r1_day', 'json']
['hursmax', 'hursmin', 'pr', 'rsds', 'sfcWindmax', 'tasmax', 'tasmin']


In [ ]:
%%time
# read in a few variables from one member
#)#,'tasmax','hursmin'})#,'sfcWindmax'})
#cat_subset=cat.search(variable={'pr'})#,'tasmax','hursmin'})#,'sfcWindmax'})
pr = cat.search(variable={'pr'}).to_dask(cdf_kwargs={'chunks': {'time': 365, 'lat': 691, 'lon': 886}})
tasmax = cat.search(variable={'tasmax'}).to_dask(cdf_kwargs={'chunks': {'time': 365, 'lat': 691, 'lon': 886}})
hursmin = cat.search(variable={'hursmin'}).to_dask(cdf_kwargs={'chunks': {'time': 365, 'lat': 691, 'lon': 886}})
sfcWindmax = cat.search(variable={'sfcWindmax'}).to_dask(cdf_kwargs={'chunks': {'time': 365, 'lat': 691, 'lon': 886}})

# hursmin has a different time axis than the other variables!
# could do two seperate datasets to deal with the different tima axis but not doing now.
hursmin=hursmin.assign_coords(time= pr.time)

# BoM files have height coordinate in the sfcWindmax that needs to be removed
if ACS_model == 'BOM':
    sfcWindmax=sfcWindmax.reset_coords('height', drop=True)
    tasmax=tasmax.reset_coords('height', drop=True)
    hursmin=hursmin.reset_coords({'height','level_height','model_level_number','sigma'}, drop=True)

hursmin

## Calculate FFDI 

In [ ]:
%%time
# This is usually computed over a specific period
#pr_annual = dsets.pr.resample(time="A").sum().mean("time")
pr_annual = pr.pr.resample(time="Y").sum().\
    sel(time=slice('2015-01-01', '2035-01-01')).mean("time").\
    assign_attrs(
    units="mm a-1", long_name="Annual average rainfall")

pr_annual

In [ ]:
t1='2015-01-01'; t2='2015-12-31'
pra=pr.pr.sel(time=slice(t1, t2))
tasmaxa=tasmax.tasmax.sel(time=slice(t1,t2))
hursmina=hursmin.hursmin.sel(time=slice(t1,t2))
sfcWindmaxa=sfcWindmax.sfcWindmax.sel(time=slice(t1,t2))
#pra,tasmaxa,hursmina,sfcWindmaxa

In [ ]:
KBDI = keetch_byram_drought_index(pra, tasmaxa, pr_annual) 
DF = griffiths_drought_factor(pra, KBDI)
FFDI = mcarthur_forest_fire_danger_index(DF, tasmaxa, hursmina, sfcWindmaxa)
ds = FFDI.to_dataset(name='FFDI')

In [ ]:
%%time
FFDI.to_netcdf('/g/data/ia39/ncra/fire/'+model_name[0]+'_FFDI.nc')

In [ ]:
%%time
ds.to_zarr('/g/data/ia39/ncra/fire/'+model_name[0]+'_FFDI.zar',mode='w')

# The End  added some extra plotting to check the input info

In [ ]:
#client.shutdown()

In [ ]:
plt.subplot(2,2,1); pra.mean('time').plot()
plt.subplot(2,2,2); tasmaxa.mean('time').plot()
plt.subplot(2,2,3); hursmina.mean('time').plot()
plt.subplot(2,2,4); sfcWindmaxa.mean('time').plot()

In [ ]:
ds=xr.open_zarr('/g/data/ia39/ncra/fire/'+model_name[0]+'_FFDI.zar')
#ffdi=xr.open_zarr('/g/data/ia39/ncra/fire/'+'AGCD-05i_CSIRO_EC-Earth3_ssp370_r1i1p1f1_CCAM-v2203-SN_v1-r1_day_FFDI.zar')
ds

In [ ]:
ds.FFDI.max('time').plot()